In [1]:
data_var = '2024-02-02'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12695,2024-02-02,França Lfb Feminina,16:00,Bourges F,Charnay Bourgogne Sud F,1.05,8.25,141.5,1.81,1.89,-17.5,2.07,1.67,EwZa0avd,0.952381,0.121212,0.552486,0.529101,0.073593,85.562,12.218681,0.142805,1.8,1.643168,0.912871,94.52,1.150,0.177764,0.154577,20.0,160.528,30.620579,0.190749,1.8,1.643168,0.912871,130.98,2.282,0.515723,0.225996,-8.0,68,74,1.39,1.77,85.772,0.000,1.094875,0.030578,0.151253,-1.65,-0.330,-0.151515,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12696,2024-02-02,França Lfb Feminina,16:00,Flammes Carolo F,Landes F,3.98,1.22,135.5,1.82,1.88,2.5,2.07,1.67,KzQkNMo9,0.251256,0.819672,0.549451,0.531915,0.070928,143.794,103.001100,0.716310,1.2,1.643168,1.369306,327.60,2.272,1.786776,0.786433,-10.0,90.020,20.143743,0.223770,2.4,1.341641,0.559017,99.96,1.366,0.275372,0.201590,33.0,60,84,5.46,1.19,0.000,0.000,0.750621,0.022933,0.151253,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12697,2024-02-02,França Lfb Feminina,16:00,La Roche F,ESB Villeneuve F,6.57,1.09,145.5,1.83,1.87,12.5,1.97,1.71,QmejL0GL,0.152207,0.917431,0.546448,0.534759,0.069638,269.760,195.046733,0.723038,1.2,1.643168,1.369306,206.70,4.126,2.586896,0.626974,-14.0,131.842,66.367368,0.503386,1.8,1.643168,0.912871,241.40,1.672,0.753140,0.450443,50.0,78,85,2.65,2.84,0.000,0.000,1.011735,0.015289,0.099917,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12698,2024-02-02,França Lfb Feminina,16:00,Lattes Montpellier F,Union Hainaut F,1.05,8.65,140.5,1.80,1.88,-17.5,2.04,1.68,Stw4aJg2,0.952381,0.115607,0.555556,0.531915,0.067988,145.022,72.391650,0.499177,0.6,1.341641,2.236068,175.50,1.972,0.927157,0.470161,-30.0,343.750,324.374633,0.943635,0.6,1.341641,2.236068,151.80,5.256,4.732508,0.900401,-78.0,90,55,1.95,2.76,0.000,0.000,1.108044,0.030744,0.136859,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
12699,2024-02-02,Geórgia Superliga,12:00,Batumi,Gurjaani Delta,2.02,1.72,158.5,1.86,1.84,1.5,1.87,1.83,QHQ5x7En,0.495050,0.581395,0.537634,0.543478,0.076445,167.964,86.584623,0.515495,1.2,1.643168,1.369306,169.74,2.056,0.969809,0.471697,-14.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,82,60,2.07,1.79,0.000,0.000,0.113440,0.007644,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,2024-02-02,Eua Ncaa,00:00,Cal St. Bakersfield,Long Beach State,2.28,1.67,144.5,1.91,1.91,1.5,2.00,1.71,OzTKigTf,0.438596,0.598802,0.523560,0.523560,0.037399,178.946,103.613802,0.579023,1.8,1.643168,0.912871,128.00,2.678,1.367267,0.510555,19.0,147.668,25.744412,0.174340,1.2,1.643168,1.369306,118.95,2.090,0.358678,0.171616,-14.0,50,65,2.56,1.83,165.074,141.194,0.218398,0.000000,0.110545,0.05,0.010,128.000000,0.0,0.0,0.0,-0.27,-0.054,-12.407407,0.559491,0.6,0.040509
12834,2024-02-02,Eua Ncaa,00:00,CS Fullerton,Hawaii,1.77,2.10,128.5,1.91,1.91,-3.5,2.10,1.67,W0WSkXc7,0.564972,0.476190,0.523560,0.523560,0.041162,148.840,69.257225,0.465313,1.8,1.643168,0.912871,105.40,2.192,0.833679,0.380328,18.0,116.982,40.765846,0.348480,1.2,1.6

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12707,13:30,Polônia Liga De Basquete,Lancut,Dziki Warszawa,1.75,Back Home
12774,05:30,Austrália Nbl,Adelaide 36ers,Sydney Kings,2.04,Back Home
12776,08:35,China Cba,Fujian,Tianjin,1.95,Back Home
12819,00:30,Eua Ncaa,USC,Oregon,2.18,Back Home
